In [ ]:
from pathlib import Path
from tqdm.auto import tqdm
import os
import re

In [ ]:
analytics_path = Path("./iDevice Analytics")
analytics_path.mkdir(exist_ok=True)

battery_stats_path = Path("./Battery Stats")
battery_stats_path.mkdir(exist_ok=True)

In [ ]:
final_output = ""
files = sorted([f for f in os.listdir(analytics_path) if f.endswith('.ips.ca.synced')])
start_date = ".".join(reversed(files[0][10:-21].split("-")))
end_date = ".".join(reversed(files[-1][10:-21].split("-")))

In [ ]:
for file in tqdm(files):
    date = ".".join(reversed(file[10:-21].split("-")))
    
    with open(analytics_path / file, "r") as f:
        data = f.read()

    DeviceModel = re.search(r'"os_version":\s*"(.*?)"', data).group(1).split(" ")[0]
    MaximumFCC = int(re.search(r'(?<=\"last_value_MaximumFCC\"\:).*?.\d+', data).group(0))
    NominalChargeCapacity = int(re.search(r'(?<="last_value_NominalChargeCapacity":).*?\d+', data).group(0))
    CycleCount = int(re.search(r'(?<=\"last_value_CycleCount\"\:).*?.\d+', data).group(0))
    RealCapacity = round(NominalChargeCapacity * 100 / MaximumFCC, 2)

    output = (
        f"Date: {date}\n"
        f"Your {DeviceModel} battery's real stats:\n"
        f"Maximum FCC (Original Maximum Capacity): {MaximumFCC} mAh\n"
        f"Nominal Charge Capacity (Current Maximum Capacity): {NominalChargeCapacity} mAh\n"
        f"Cycle Count (Number of Complete Charge Cycles): {CycleCount} cycles\n"
        f"Relative to when it was new, your battery capacity is {RealCapacity}%.\n\n"
    )
    
    final_output += output

In [ ]:
with open(battery_stats_path / f"battery_stats_{start_date}-{end_date}.txt", "w") as f:
    f.write(final_output.strip())